In [ ]:
import pandas as pd
import mysql.connector

In [6]:
SOURCE_HOST = "localhost"    
SOURCE_USER = "adados"    
SOURCE_PASSWORD = "12345"

In [7]:
conn = mysql.connector.connect(host=SOURCE_HOST, 
                            port="3306", 
                            user=SOURCE_USER, 
                            password=SOURCE_PASSWORD, 
                            autocommit=True)
if not conn.is_connected():
    print("Erro de Conexão")
    cursor = None
else:
    print("Conexão com Sucesso")
    cursor = conn.cursor()
    

Conexão com Sucesso


In [8]:
# Massa de dados para a tabela dados.cliente
clientes = [
    {"nome": "Ana Silva", "cidade": "São Paulo"},
    {"nome": "Bruno Oliveira", "cidade": "Rio de Janeiro"},
    {"nome": "Carla Souza", "cidade": "Belo Horizonte"},
    {"nome": "Diego Santos", "cidade": "Curitiba"},
    {"nome": "Elena Costa", "cidade": "Porto Alegre"},
    {"nome": "Fábio Pereira", "cidade": "Salvador"},
    {"nome": "Giovanna Lima", "cidade": "Recife"},
    {"nome": "Henrique Rocha", "cidade": "Brasília"},
    {"nome": "Isabela Martins", "cidade": "Fortaleza"},
    {"nome": "João Ferreira", "cidade": "Manaus"},
    {"nome": "Karina Alves", "cidade": "Goiânia"},
    {"nome": "Lucas Barbosa", "cidade": "Belém"},
    {"nome": "Mariana Ribeiro", "cidade": "Florianópolis"},
    {"nome": "Natan Gomes", "cidade": "Vitória"},
    {"nome": "Olívia Castro", "cidade": "Natal"},
    {"nome": "Paulo Mendes", "cidade": "João Pessoa"},
    {"nome": "Quitéria Duarte", "cidade": "Maceió"},
    {"nome": "Ricardo Nunes", "cidade": "Teresina"},
    {"nome": "Sabrina Vieira", "cidade": "Campo Grande"},
    {"nome": "Tiago Cardoso", "cidade": "Cuiabá"}
]

In [9]:
#Exemplo 1 - Inserção de Dados
for c in clientes:
    cursor.execute(
        "INSERT INTO dados.cliente (nome, cidade) VALUES (%s, %s)",
        (c["nome"], c["cidade"])
    )

# utilizado para gravar em lote de registros
#  quando o autocommit da conexão estiver False
conn.commit() 
# Fechar o Cursor e Liberar recursos   
cursor.close()
# Fechar a Conexão com o banco de dados
# conn.close()

True

In [10]:
#Exemplo 2 - Inserção de Dados
cursor = conn.cursor()

sql = "INSERT INTO dados.cliente (nome, cidade) VALUES (%s, %s)"

valores = [(c["nome"], c["cidade"]) for c in clientes]

cursor.executemany(sql, valores)

conn.commit()
cursor.close()

True

In [11]:
# Exemplo 3 - Inserção de Dados 
# Usando Dataframe
cursor = conn.cursor()

df = pd.DataFrame(clientes)

sql = "INSERT INTO dados.cliente (nome, cidade) VALUES (%s, %s)"

valores = list(df[["nome", "cidade"]].itertuples(index=False, name=None))

cursor.executemany(sql, valores)

conn.commit()
cursor.close()


True

In [12]:
# Exemplo 4 - Inserção de Dados 
# Usando iterrows

cursor = conn.cursor()
df = pd.DataFrame(clientes)

for _, row in df.iterrows():
    cursor.execute(
        "INSERT INTO dados.cliente (nome, cidade) VALUES (%s, %s)",
        (row["nome"], row["cidade"])
    )

conn.commit()
cursor.close()

True

In [13]:
# Exemplo 5 - Inserção de Dados 
cursor = conn.cursor()
df = pd.DataFrame(clientes)
sql = "INSERT INTO dados.cliente (nome, cidade) VALUES (%s, %s)"

cursor.executemany(sql, df[["nome", "cidade"]].values.tolist())

conn.commit()
cursor.close()

True

In [21]:
# Exemplo 1 - Delete passando parametro
# o parametro pode ser também uma variavel
cursor = conn.cursor()
query = "DELETE FROM dados.cliente WHERE cidade = %s"

cursor.execute(query, ("São Paulo",))
conn.commit()
cursor.close()
print("✅ Linhas deletadas:", cursor.rowcount)


✅ Linhas deletadas: 5


In [22]:
# Exemplo 2 - Delete dataframe como parametro

cursor = conn.cursor()

df_delete = pd.DataFrame({
    "cidade": ["Curitiba", "Porto Alegre"]
})

query = "DELETE FROM dados.cliente WHERE cidade = %s"

dados = list(df_delete[["cidade"]].itertuples(index=False, name=None))

cursor.executemany(query, dados)
conn.commit()
cursor.close()

print("✅ Linhas deletadas:", cursor.rowcount)


✅ Linhas deletadas: 10


In [14]:
# Exemplo 1 - Update como parametro

cursor = conn.cursor()

query = "UPDATE dados.cliente SET cidade = %s WHERE cidade = %s"
cursor.execute(query, ("Vitória", "Brasília", ))
conn.commit()
cursor.close()

print("✅ Atualizados:", cursor.rowcount)

✅ Atualizados: 5


In [15]:
# Exemplo 2 - Update como dataframe

cursor = conn.cursor()

df_upd = pd.DataFrame({
    "cidade": ["Goiânia", "Vitória", "Teresina"]
})

dados = list(df_upd[["cidade"]].itertuples(index=False, name=None))

query = "UPDATE dados.cliente SET cidade = Null WHERE cidade = %s"

cursor.executemany(query, dados)
conn.commit()
cursor.close()

print("✅ Atualizados:", cursor.rowcount)

✅ Atualizados: 20


In [16]:
# Exemplo 1 - Consultando banco de dados

cursor = conn.cursor()

# 2. Executa a consulta
query = "SELECT * FROM dados.cliente"
cursor.execute(query)

# 3. Pega todos os resultados e os nomes das colunas
dados = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]

# 4. Cria o DataFrame usando os dados e colunas capturados
df_select = pd.DataFrame(dados, columns=colunas)

# 5. Exibe, fecha e limpa
print(df_select)
cursor.close()
del df_select

     id             nome          cidade
0     1        Ana Silva       São Paulo
1     2   Bruno Oliveira  Rio de Janeiro
2     3      Carla Souza  Belo Horizonte
3     4     Diego Santos        Curitiba
4     5      Elena Costa    Porto Alegre
..  ...              ...             ...
95   96     Paulo Mendes     João Pessoa
96   97  Quitéria Duarte          Maceió
97   98    Ricardo Nunes            None
98   99   Sabrina Vieira    Campo Grande
99  100    Tiago Cardoso          Cuiabá

[100 rows x 3 columns]


In [17]:
# Exemplo 3 - Consultando banco de dados com filtro
# PARTE 1
cursor = conn.cursor()
# 1. Seu DataFrame com os filtros
df_filtros = pd.DataFrame({
    "cidade": ["Recife", "Rio de Janeiro"]
})
# 2. Transforma em tupla (O SQL precisa do formato entre parênteses)
lista_cidades = str(tuple(df_filtros["cidade"].tolist()))
# 3. A Query usa o formato de string, mas injetamos a tupla de forma segura
# Note que não usamos aspas em volta do %s
query = f"SELECT * FROM dados.cliente WHERE cidade IN {lista_cidades}"
print(query)
# Executando a string Query
cursor.execute(query)


SELECT * FROM dados.cliente WHERE cidade IN ('Recife', 'Rio de Janeiro')


In [18]:
# PARTE 2
# Carrega todos os dados lidos na lista dados
dados = cursor.fetchall()
# define o nome das colunas
colunas = [desc[0] for desc in cursor.description]
# cria o dataframe
df_select = pd.DataFrame(dados, columns=colunas)
# 5. Resultado e Limpeza
print(df_select)
cursor.close()
del df_select

   id            nome          cidade
0   2  Bruno Oliveira  Rio de Janeiro
1   7   Giovanna Lima          Recife
2  22  Bruno Oliveira  Rio de Janeiro
3  27   Giovanna Lima          Recife
4  42  Bruno Oliveira  Rio de Janeiro
5  47   Giovanna Lima          Recife
6  62  Bruno Oliveira  Rio de Janeiro
7  67   Giovanna Lima          Recife
8  82  Bruno Oliveira  Rio de Janeiro
9  87   Giovanna Lima          Recife


In [19]:
# Exemplo 4 - Consultando banco de dados com filtro
# PARTE 1
cursor = conn.cursor()
# 1. Seu DataFrame com os filtros
df_filtros = pd.DataFrame({
    "cidades": ["Recife", "Rio de Janeiro"]
})

# 2. Prepara os parâmetros
# Transformamos a coluna do DF em uma lista simples de Python
lista_cidades = df_filtros["cidades"].tolist()

# Criamos uma string com o número correto de placeholders (%s)
# Para 2 cidades, gerará: "%s, %s"
placeholders = ', '.join(['%s'] * len(lista_cidades))

# 3. Monta a Query dinâmica
query = f"SELECT * FROM dados.cliente WHERE cidade IN ({placeholders})"
print(query)

SELECT * FROM dados.cliente WHERE cidade IN (%s, %s)


In [20]:
# Exemplo 4 - Consultando banco de dados com filtro
# PARTE 2
# 4. Executa e transforma em DataFrame
cursor = conn.cursor()
cursor.execute(query, lista_cidades) # Passamos a lista como segundo argumento

dados = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]

df_select = pd.DataFrame(dados, columns=colunas)

# 5. Resultado e Limpeza
print(df_select)
cursor.close()
del df_select

   id            nome          cidade
0   2  Bruno Oliveira  Rio de Janeiro
1   7   Giovanna Lima          Recife
2  22  Bruno Oliveira  Rio de Janeiro
3  27   Giovanna Lima          Recife
4  42  Bruno Oliveira  Rio de Janeiro
5  47   Giovanna Lima          Recife
6  62  Bruno Oliveira  Rio de Janeiro
7  67   Giovanna Lima          Recife
8  82  Bruno Oliveira  Rio de Janeiro
9  87   Giovanna Lima          Recife
